<a href="https://colab.research.google.com/github/danialebrat/Stock_Prediction_khodro_group/blob/main/Codes/prediction_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler, minmax_scale

In [ ]:
# loading the model
model = load_model("/content/drive/MyDrive/Stock Prediction/Stock_model.h5")

In [ ]:
def printTransactions(m, k, d, name, owned, prices):
    
    """

    INPUT: 

    m - the amount of money you could spend that day.
    k - the number of different stocks available for buying or selling.
    d - the number of remaining days for trading stocks.

    k lines follow, each in the following format: name owned prices
    name - the name of the stock (a string).
    owned - the number of shares you own of that stock.
    prices - 5 space separated numbers representing the stock's price for the last 5 days. last number is the current stock price.

    OUTPUT:
    Output N - number of transactions, 0 if not any
    output N lines containing the name of the stock, BUY or SELL, and the number of shares to buy or sell.
    """
    # list of Potential buy or sell
    potential_sale = []
    potential_buy = []

    # Knapstack_input :(name of stock, current price, balance = future - current)
    capacity = m
    knapstack_Input = []

    

    for i in range(k):

        # pridicting the future price of each stock
        future_price = predict_future(prices[i])

        # ballance is > 0 if the price increases next day
        ballance = future_price - prices[i][-1]

        # if the price is going to decrease (ballance < 0)
        if ballance < 0:
          if owned[i] > 0:
            # Sell each stock that is going to fall down the next day
            sale = ('{} SELL {}'.format(name[i], owned[i]))
            potential_sale.append(sale)

            
        else:
          # if the price is going to INCREASE (ballance > 0)
          # add it to knapstack_Input
          # (name, current price, balance)

          # unbounded knapstack is for int values
          # therefore, we multiply the prices by 100 to get int values
          row = (name[i], int(prices[i][-1]*100), int(round(ballance, 2)*100))
          knapstack_Input.append(row)

    # solving the unbounded_knapsack
    buy = unbounded_knapsack(knapstack_Input, int(capacity*100))
    potential_buy = buy_list(buy)

    # print the output using potential_buy and potential_sale
    print_output(potential_buy, potential_sale)

   

In [ ]:
def buy_list(buy):
  """
  creates potential buy list
  which stock to buy? how many?
  """

  potential_buy = []
  unique_items = np.unique(np.array(buy))
  for i in unique_items:
    count = np.count_nonzero(np.array(buy) == i)
    b = ('{} BUY {}'.format(i, count))
    potential_buy.append(b)
  
  return potential_buy 



In [ ]:
def predict_future(prices):
    

    price_list = []

    original_data = np.array(prices)

    # create the input for the model (1,5) --> (5,1)
    segment = original_data.reshape((-1, 1))
    
    # scale the segment in (0, 1)
    scaler = MinMaxScaler()
    arr = scaler.fit_transform(segment)

    # the model input take (N, 5, 1) but the segment is (5,1)
    # Therefore, we add the segment to a list to have (1, 5, 1)
    price_list.append(arr)

    # predicting the future price
    future_price = model.predict(price_list)

    # reverse the scale to the original price
    predicted_price = scaler.inverse_transform(future_price[0])

    return predicted_price[0][0]

In [ ]:
def print_output(potential_buy, potential_sale):

  # if potential_buy and potential_sale are empty
  # we don't have any transaction
  if len(potential_buy)+len(potential_sale) == 0:
    print(0)

  else:
    # if not, print each list that is not empty
    number_of_transaction = len(potential_buy)+len(potential_sale)
    print(number_of_transaction)

    if len(potential_buy) > 0:
      for i in potential_buy:
        print(i)
    
    if len(potential_sale) > 0:
      for i in potential_sale:
        print(i)


In [ ]:
def unbounded_knapsack(knapstack_Input, capacity):

    """
    unbounded_knapsack solution using dynamic programming
    """

    stock = []
    weights = [] 
    profits = []

    #(name, current price, balance)
    for name, weight, profit in knapstack_Input:
      weights.append(weight)
      profits.append(profit)
      stock.append(name)

    
    n = len(profits)
    table = [[0] * (capacity + 1) for i in range(n)]

    # When capacity is 0, none of the item can be selected
    for i in range(n):
        table[i][0] = 0

    # When only 1 item is considered, get the profits using that item for every capapcity 'c'
    for c in range(capacity + 1):
        table[0][c] = (c // weights[0]) * profits[0]

    # For each item, check for every possible capacities
    for i in range(1, n):
        for c in range(1, capacity + 1):
            if (weights[i] <= c):
                table[i][c] = max(table[i - 1][c], profits[i] + table[i][c - weights[i]])
            else:
                table[i][c] = table[i - 1][c]

    # we find the maximum profit, now we need to find the items
    buy = pick_selected_items(table, weights, profits, stock)

    return buy

In [ ]:
def pick_selected_items(table, weights, profits, name):
    
    """
    finding the items for maximum profit
    """

    buy = []

    i, c = len(table)-1, len(table[0])-1
    total_profit = table[i][c]
    while (i-1 >= 0 and table[i][c] >= 0):
        if (table[i][c] != table[i - 1][c]):
            buy.append(name[i])
            c -= weights[i]
            total_profit -= profits[i]
        else:
            i -= 1

    while (total_profit > 0):
        buy.append(name[0])
        total_profit -= profits[0]

    return buy

In [ ]:
def readInputAndRun():
    # reading input
    line = input().split()

    # m = 100, k = 10, d = 20
    m, k, d = float(line[0]), int(line[1]), int(line[2])
    name = []
    owned = []
    prices = []

    for i in range(k):
        line = input().split()
        # [0] is the name of the stock
        name.append(line[0])
        # [1] is the number of shares we owned from the stock
        owned.append(int(line[1]))

        # [2:] prices for the 5 days
        prices.append(list(map(float, line[2:])))    

    printTransactions(m, k, d, name, owned, prices)

In [ ]:
if __name__ == "__main__":


  readInputAndRun()

#Input

# 100 10 20
# CAL 4 121.83 122.26 123.94 122.11 120.58
# UCB 3 52.34 50.89 47.62 51.16 52.4
# RIT 1 100.21 99.33 102.87 110.63 110.72
# UCLA 1 12.5 27.22 9.32 16.07 3.86
# USC 4 249.58 244.24 242.56 245.13 245.35
# UFL 2 20.21 19.77 21.34 20.21 21.17
# UMAD 2 109.39 120.49 131.35 127.97 121.38
# RICE 2 126.3 128 129.08 129.28 124.44
# UMD 3 102.96 103.62 98.22 96.6 99.1
# UCSC 2 213.77 193.51 178.53 180.08 208.29

100 10 20
CAL 4 121.83 122.26 123.94 122.11 120.58
UCB 3 52.34 50.89 47.62 51.16 52.4
IT 1 100.21 99.33 102.87 110.63 110.72
UCLA 1 12.5 27.22 9.32 16.07 3.86
USC 4 249.58 244.24 242.56 245.13 245.35
FL 2 20.21 19.77 21.34 20.21 21.17
UMAD 2 109.39 120.49 131.35 127.97 121.38
RICE 2 126.3 128 129.08 129.28 124.44
UMD 3 102.96 103.62 98.22 96.6 99.1
UCSC 2 213.77 193.51 178.53 180.08 208.29
6
UCLA BUY 25
UCB SELL 3
IT SELL 1
USC SELL 4
FL SELL 2
UCSC SELL 2
